In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint

from sklearn.preprocessing import LabelEncoder

# 저장된 딥러닝 모델을 복구하는 함수.
from keras.models import load_model

# epoch마다 모델을 저장하는 함수.
from keras.callbacks import ModelCheckpoint

# 더이상 성능 향상이 이루어지지 않는다면 조기 중단시킬 수 있는 함수.
from keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold 

import warnings
warnings.filterwarnings('ignore')

# GPU 사용 초기화 및 할당.
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [2]:
# Seed 값 설정.
np.random.seed(3)
tf.random.set_seed(3)

# 데이터 읽기.
df = pd.read_csv('data/breast_cancer.csv')
df.head(10)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,malignant
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,malignant
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,malignant
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,malignant
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,malignant
5,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,0.07613,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,malignant
6,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,0.05742,...,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368,malignant
7,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,0.07451,...,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510,malignant
8,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,0.07389,...,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720,malignant
9,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,0.08243,...,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750,malignant


In [3]:
# 데이터 정보 확인.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              454 non-null    float64
 1   mean texture             454 non-null    float64
 2   mean perimeter           454 non-null    float64
 3   mean area                454 non-null    float64
 4   mean smoothness          454 non-null    float64
 5   mean compactness         454 non-null    float64
 6   mean concavity           454 non-null    float64
 7   mean concave points      454 non-null    float64
 8   mean symmetry            454 non-null    float64
 9   mean fractal dimension   454 non-null    float64
 10  radius error             454 non-null    float64
 11  texture error            454 non-null    float64
 12  perimeter error          454 non-null    float64
 13  area error               454 non-null    float64
 14  smoothness error         4

In [4]:
# 결측 재확인.
df.isna().sum()

mean radius                0
mean texture               0
mean perimeter             0
mean area                  0
mean smoothness            0
mean compactness           0
mean concavity             0
mean concave points        0
mean symmetry              0
mean fractal dimension     0
radius error               0
texture error              0
perimeter error            0
area error                 0
smoothness error           0
compactness error          0
concavity error            0
concave points error       0
symmetry error             0
fractal dimension error    0
worst radius               0
worst texture              0
worst perimeter            0
worst area                 0
worst smoothness           0
worst compactness          0
worst concavity            0
worst concave points       0
worst symmetry             0
worst fractal dimension    0
target                     0
dtype: int64

In [5]:
# Data shape
# 31개의 컬럼으로 구성.
df.shape

(454, 31)

In [6]:
# 데이터 분류.
# 30개의 특성데이터와 1개의 타겟데이터로 분할.
x = df.drop('target', axis=1)
y = df['target']
# display(x)
# display(y)

In [7]:
# 문자열을 숫자로 변환.
encoder1 = LabelEncoder()
encoder1.fit(y)
y = encoder1.transform(y)
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,

In [8]:
# 모델 설정.
model = Sequential()

# 30개의 노드로 이루어진 입력층과 2개의 은닉층 설정.
model.add(Dense(60, input_dim=30, activation='relu'))
model.add(Dense(40, activation='relu'))

# 타겟 데이터가 이진형 범주형 변수이므로 활성화 함수로 sigmoid 통과.
model.add(Dense(1, activation='sigmoid'))
    
# 모델 컴파일.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 참고.
# 가장 기본적인 모델 저장 방법.
# model.save('test1.h5') 이용.

# epoch 마다 저장하는 방식으로 전환.
# 저장될 경로와 파일명.
model_path = 'models/{epoch}-{val_loss}.hdf5'

# 기준은 val_loss 외의 var_accrucy를 줘도 되나, 다소의 문제가 있으므로 통상적으로 'val_loss'를 지정.
# 저장 조건1 : 'val_loss'를 기준으로 이전 값들보다 높은 수치는 저장하지 않으며 이전 값들 보다 낮은 수치만 해당 경로로 저장.
checkpointer = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)

# 저장 조건2 : 'val_loss' 값이 지정된 수(100) 만큼 학습하여도 나아질 기미가 없다면 학습을 중단.
# patience : 지정해준 숫자만큼 동안 성능 향상이 이루어지지 않았다면 중단.
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=100)

# 평가 지표 : 'val_loss' : 검증용 손실값.
# 모델 실행.
# validation_split : x, y 값 중 20% 만큼은 검증용으로 활용.
history = model.fit(x, y, validation_split=0.2, epochs=1000, batch_size=200, verbose=1, 
                    callbacks=[checkpointer, early_stopping_callback] )

Epoch 1/1000
2/2 [==============================] - 1s 298ms/step - loss: 114.3484 - accuracy: 0.5510 - val_loss: 55.2283 - val_accuracy: 0.7253

Epoch 00001: val_loss improved from inf to 55.22828, saving model to models\1-55.22827911376953.hdf5
Epoch 2/1000
2/2 [==============================] - 0s 29ms/step - loss: 79.3046 - accuracy: 0.5526 - val_loss: 33.5102 - val_accuracy: 0.7253

Epoch 00002: val_loss improved from 55.22828 to 33.51019, saving model to models\2-33.51019287109375.hdf5
Epoch 3/1000
2/2 [==============================] - 0s 28ms/step - loss: 49.5116 - accuracy: 0.5410 - val_loss: 12.0632 - val_accuracy: 0.6374

Epoch 00003: val_loss improved from 33.51019 to 12.06322, saving model to models\3-12.06321907043457.hdf5
Epoch 4/1000
2/2 [==============================] - 0s 29ms/step - loss: 16.1196 - accuracy: 0.4273 - val_loss: 19.6655 - val_accuracy: 0.2747

Epoch 00004: val_loss did not improve from 12.06322
Epoch 5/1000
2/2 [==============================] - 0s 30

Epoch 40/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.2656 - accuracy: 0.9219 - val_loss: 0.3081 - val_accuracy: 0.9341

Epoch 00040: val_loss did not improve from 0.30283
Epoch 41/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.2392 - accuracy: 0.9306 - val_loss: 0.2970 - val_accuracy: 0.9341

Epoch 00041: val_loss improved from 0.30283 to 0.29696, saving model to models\41-0.29695889353752136.hdf5
Epoch 42/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.2527 - accuracy: 0.9256 - val_loss: 0.2851 - val_accuracy: 0.8681

Epoch 00042: val_loss improved from 0.29696 to 0.28513, saving model to models\42-0.28513476252555847.hdf5
Epoch 43/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.2460 - accuracy: 0.9219 - val_loss: 0.2968 - val_accuracy: 0.8462

Epoch 00043: val_loss did not improve from 0.28513
Epoch 44/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.2520 - accuracy: 0.9114 - val_loss: 

2/2 [==============================] - 0s 29ms/step - loss: 0.1820 - accuracy: 0.9222 - val_loss: 0.1868 - val_accuracy: 0.9341

Epoch 00077: val_loss did not improve from 0.18241
Epoch 78/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.1734 - accuracy: 0.9308 - val_loss: 0.1862 - val_accuracy: 0.9341

Epoch 00078: val_loss did not improve from 0.18241
Epoch 79/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.1685 - accuracy: 0.9289 - val_loss: 0.1809 - val_accuracy: 0.9341

Epoch 00079: val_loss improved from 0.18241 to 0.18094, saving model to models\79-0.1809379607439041.hdf5
Epoch 80/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.1710 - accuracy: 0.9322 - val_loss: 0.1844 - val_accuracy: 0.9341

Epoch 00080: val_loss did not improve from 0.18094
Epoch 81/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.1580 - accuracy: 0.9356 - val_loss: 0.1846 - val_accuracy: 0.9341

Epoch 00081: val_loss did not improve fr

2/2 [==============================] - 0s 29ms/step - loss: 0.1667 - accuracy: 0.9374 - val_loss: 0.1788 - val_accuracy: 0.9451

Epoch 00118: val_loss did not improve from 0.16447
Epoch 119/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.1694 - accuracy: 0.9343 - val_loss: 0.1895 - val_accuracy: 0.9341

Epoch 00119: val_loss did not improve from 0.16447
Epoch 120/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.1738 - accuracy: 0.9258 - val_loss: 0.1848 - val_accuracy: 0.9451

Epoch 00120: val_loss did not improve from 0.16447
Epoch 121/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.1661 - accuracy: 0.9326 - val_loss: 0.1673 - val_accuracy: 0.9451

Epoch 00121: val_loss did not improve from 0.16447
Epoch 122/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.1675 - accuracy: 0.9219 - val_loss: 0.1723 - val_accuracy: 0.9451

Epoch 00122: val_loss did not improve from 0.16447
Epoch 123/1000
2/2 [====================

2/2 [==============================] - 0s 30ms/step - loss: 0.1683 - accuracy: 0.9224 - val_loss: 0.1577 - val_accuracy: 0.9451

Epoch 00159: val_loss improved from 0.16084 to 0.15769, saving model to models\159-0.15768581628799438.hdf5
Epoch 160/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.1690 - accuracy: 0.9221 - val_loss: 0.1653 - val_accuracy: 0.9451

Epoch 00160: val_loss did not improve from 0.15769
Epoch 161/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.1521 - accuracy: 0.9308 - val_loss: 0.1850 - val_accuracy: 0.9341

Epoch 00161: val_loss did not improve from 0.15769
Epoch 162/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.1541 - accuracy: 0.9374 - val_loss: 0.1931 - val_accuracy: 0.9231

Epoch 00162: val_loss did not improve from 0.15769
Epoch 163/1000
2/2 [==============================] - 0s 28ms/step - loss: 0.1603 - accuracy: 0.9378 - val_loss: 0.1572 - val_accuracy: 0.9451

Epoch 00163: val_loss improved fro

...

2/2 [==============================] - 0s 31ms/step - loss: 0.0986 - accuracy: 0.9751 - val_loss: 0.0661 - val_accuracy: 0.9780

Epoch 00816: val_loss did not improve from 0.06525
Epoch 817/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.1206 - accuracy: 0.9486 - val_loss: 0.1101 - val_accuracy: 0.9560

Epoch 00817: val_loss did not improve from 0.06525
Epoch 818/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.1073 - accuracy: 0.9630 - val_loss: 0.0981 - val_accuracy: 0.9560

Epoch 00818: val_loss did not improve from 0.06525
Epoch 819/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.0916 - accuracy: 0.9681 - val_loss: 0.0650 - val_accuracy: 0.9780

Epoch 00819: val_loss improved from 0.06525 to 0.06502, saving model to models\819-0.06502198427915573.hdf5
Epoch 820/1000
2/2 [==============================] - 0s 31ms/step - loss: 0.0933 - accuracy: 0.9606 - val_loss: 0.1219 - val_accuracy: 0.9560

Epoch 00820: val_loss did not impr

2/2 [==============================] - 0s 30ms/step - loss: 0.0900 - accuracy: 0.9715 - val_loss: 0.0634 - val_accuracy: 0.9890

Epoch 00857: val_loss did not improve from 0.06220
Epoch 858/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.1001 - accuracy: 0.9541 - val_loss: 0.1238 - val_accuracy: 0.9560

Epoch 00858: val_loss did not improve from 0.06220
Epoch 859/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.1025 - accuracy: 0.9556 - val_loss: 0.0625 - val_accuracy: 0.9890

Epoch 00859: val_loss did not improve from 0.06220
Epoch 860/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.1181 - accuracy: 0.9451 - val_loss: 0.0851 - val_accuracy: 0.9670

Epoch 00860: val_loss did not improve from 0.06220
Epoch 861/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.0842 - accuracy: 0.9678 - val_loss: 0.0704 - val_accuracy: 0.9780

Epoch 00861: val_loss did not improve from 0.06220
Epoch 862/1000
2/2 [====================


Epoch 00898: val_loss did not improve from 0.06041
Epoch 899/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.0880 - accuracy: 0.9661 - val_loss: 0.0738 - val_accuracy: 0.9670

Epoch 00899: val_loss did not improve from 0.06041
Epoch 900/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.0806 - accuracy: 0.9748 - val_loss: 0.0861 - val_accuracy: 0.9670

Epoch 00900: val_loss did not improve from 0.06041
Epoch 901/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.0829 - accuracy: 0.9680 - val_loss: 0.0688 - val_accuracy: 0.9780

Epoch 00901: val_loss did not improve from 0.06041
Epoch 902/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.0820 - accuracy: 0.9623 - val_loss: 0.0972 - val_accuracy: 0.9560

Epoch 00902: val_loss did not improve from 0.06041
Epoch 903/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.0914 - accuracy: 0.9696 - val_loss: 0.0729 - val_accuracy: 0.9670

Epoch 00903: val_loss di

2/2 [==============================] - 0s 82ms/step - loss: 0.0977 - accuracy: 0.9665 - val_loss: 0.1172 - val_accuracy: 0.9560

Epoch 00940: val_loss did not improve from 0.05843
Epoch 941/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0881 - accuracy: 0.9660 - val_loss: 0.0603 - val_accuracy: 0.9890

Epoch 00941: val_loss did not improve from 0.05843
Epoch 942/1000
2/2 [==============================] - 0s 33ms/step - loss: 0.1028 - accuracy: 0.9521 - val_loss: 0.0946 - val_accuracy: 0.9560

Epoch 00942: val_loss did not improve from 0.05843
Epoch 943/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.0830 - accuracy: 0.9693 - val_loss: 0.0715 - val_accuracy: 0.9780

Epoch 00943: val_loss did not improve from 0.05843
Epoch 944/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.0796 - accuracy: 0.9698 - val_loss: 0.1124 - val_accuracy: 0.9560

Epoch 00944: val_loss did not improve from 0.05843
Epoch 945/1000
2/2 [====================

2/2 [==============================] - 0s 34ms/step - loss: 0.1096 - accuracy: 0.9396 - val_loss: 0.0862 - val_accuracy: 0.9560

Epoch 00982: val_loss did not improve from 0.05574
Epoch 983/1000
2/2 [==============================] - 0s 34ms/step - loss: 0.0936 - accuracy: 0.9604 - val_loss: 0.0662 - val_accuracy: 0.9780

Epoch 00983: val_loss did not improve from 0.05574
Epoch 984/1000
2/2 [==============================] - 0s 32ms/step - loss: 0.1171 - accuracy: 0.9519 - val_loss: 0.1046 - val_accuracy: 0.9560

Epoch 00984: val_loss did not improve from 0.05574
Epoch 985/1000
2/2 [==============================] - 0s 29ms/step - loss: 0.0843 - accuracy: 0.9743 - val_loss: 0.0615 - val_accuracy: 0.9780

Epoch 00985: val_loss did not improve from 0.05574
Epoch 986/1000
2/2 [==============================] - 0s 30ms/step - loss: 0.0940 - accuracy: 0.9469 - val_loss: 0.1275 - val_accuracy: 0.9670

Epoch 00986: val_loss did not improve from 0.05574
Epoch 987/1000
2/2 [====================

In [9]:
# Best모델 불러오기.
best_model = load_model('models/994-0.05470036342740059.hdf5')

# Train data로 모델 성능평가.
# 학습한 데이터를 이용한 모델 성능 평가이므로 당연히 높은 수치를 보임.
best_model.evaluate(x, y)[1]

15/15 [==============================] - 0s 1ms/step - loss: 0.0867 - accuracy: 0.9626


0.9625550508499146

In [10]:
# 예측할 데이터를 읽기.
df_test = pd.read_csv('data/breast_cancer_new.csv')
df_test.head(10)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,12.620,17.15,80.62,492.9,0.08583,0.05430,0.02966,0.02272,0.1799,0.05826,...,14.34,22.15,91.62,633.5,0.1225,0.1517,0.18870,0.09851,0.3270,0.07330
1,13.380,30.72,86.34,557.2,0.09245,0.07426,0.02819,0.03264,0.1375,0.06016,...,15.05,41.61,96.69,705.6,0.1172,0.1421,0.07003,0.07763,0.2196,0.07675
2,11.630,29.29,74.87,415.1,0.09357,0.08574,0.07160,0.02017,0.1799,0.06166,...,13.12,38.81,86.04,527.8,0.1406,0.2031,0.29230,0.06835,0.2884,0.07220
3,13.210,25.25,84.10,537.9,0.08791,0.05205,0.02772,0.02068,0.1619,0.05584,...,14.35,34.23,91.29,632.9,0.1289,0.1063,0.13900,0.06005,0.2444,0.06788
4,13.000,25.13,82.61,520.2,0.08369,0.05073,0.01206,0.01762,0.1667,0.05449,...,14.34,31.88,91.06,628.5,0.1218,0.1093,0.04462,0.05921,0.2306,0.06291
5,9.755,28.20,61.68,290.9,0.07984,0.04626,0.01541,0.01043,0.1621,0.05952,...,10.67,36.92,68.03,349.9,0.1110,0.1109,0.07190,0.04866,0.2321,0.07211
6,17.080,27.15,111.20,930.9,0.09898,0.11100,0.10070,0.06431,0.1793,0.06281,...,22.96,34.49,152.10,1648.0,0.1600,0.2444,0.26390,0.15550,0.3010,0.09060
7,27.420,26.27,186.90,2501.0,0.10840,0.19880,0.36350,0.16890,0.2061,0.05623,...,36.04,31.37,251.20,4254.0,0.1357,0.4256,0.68330,0.26250,0.2641,0.07427
8,14.400,26.99,92.25,646.1,0.06995,0.05223,0.03476,0.01737,0.1707,0.05433,...,15.40,31.98,100.40,734.6,0.1017,0.1460,0.14720,0.05563,0.2345,0.06464
9,11.600,18.36,73.88,412.7,0.08508,0.05855,0.03367,0.01777,0.1516,0.05859,...,12.77,24.02,82.68,495.1,0.1342,0.1808,0.18600,0.08288,0.3210,0.07863


In [11]:
# Data shape.
# 타겟 데이터를 제외한 30개의 컬럼으로 구성.
df_test.shape

(115, 30)

In [12]:
# 예측 결과를 추출.
# 115개 데이터에 대한 예측.
pred = (best_model.predict(df_test) > 0.5).astype('int32')
pred

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
    

In [13]:
# 수치 결과를 문자열로 디코딩 작업.
pred2 = encoder1.inverse_transform(pred)
pred2

array(['benign', 'benign', 'benign', 'benign', 'benign', 'benign',
       'malignant', 'malignant', 'benign', 'benign', 'benign', 'benign',
       'benign', 'benign', 'malignant', 'benign', 'benign', 'benign',
       'benign', 'benign', 'benign', 'benign', 'benign', 'benign',
       'benign', 'benign', 'benign', 'malignant', 'benign', 'benign',
       'benign', 'benign', 'benign', 'malignant', 'benign', 'malignant',
       'benign', 'malignant', 'malignant', 'benign', 'benign', 'benign',
       'benign', 'benign', 'malignant', 'malignant', 'benign',
       'malignant', 'benign', 'malignant', 'benign', 'benign', 'benign',
       'benign', 'benign', 'malignant', 'benign', 'benign', 'malignant',
       'benign', 'malignant', 'benign', 'malignant', 'malignant',
       'benign', 'benign', 'benign', 'malignant', 'benign', 'benign',
       'benign', 'benign', 'benign', 'benign', 'benign', 'benign',
       'benign', 'benign', 'benign', 'malignant', 'benign', 'malignant',
       'benign', 'beni

In [14]:
# 예측 결과 데이터를 추가.
df_test['target'] = pred2
df_test.head(10)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,12.620,17.15,80.62,492.9,0.08583,0.05430,0.02966,0.02272,0.1799,0.05826,...,22.15,91.62,633.5,0.1225,0.1517,0.18870,0.09851,0.3270,0.07330,benign
1,13.380,30.72,86.34,557.2,0.09245,0.07426,0.02819,0.03264,0.1375,0.06016,...,41.61,96.69,705.6,0.1172,0.1421,0.07003,0.07763,0.2196,0.07675,benign
2,11.630,29.29,74.87,415.1,0.09357,0.08574,0.07160,0.02017,0.1799,0.06166,...,38.81,86.04,527.8,0.1406,0.2031,0.29230,0.06835,0.2884,0.07220,benign
3,13.210,25.25,84.10,537.9,0.08791,0.05205,0.02772,0.02068,0.1619,0.05584,...,34.23,91.29,632.9,0.1289,0.1063,0.13900,0.06005,0.2444,0.06788,benign
4,13.000,25.13,82.61,520.2,0.08369,0.05073,0.01206,0.01762,0.1667,0.05449,...,31.88,91.06,628.5,0.1218,0.1093,0.04462,0.05921,0.2306,0.06291,benign
5,9.755,28.20,61.68,290.9,0.07984,0.04626,0.01541,0.01043,0.1621,0.05952,...,36.92,68.03,349.9,0.1110,0.1109,0.07190,0.04866,0.2321,0.07211,benign
6,17.080,27.15,111.20,930.9,0.09898,0.11100,0.10070,0.06431,0.1793,0.06281,...,34.49,152.10,1648.0,0.1600,0.2444,0.26390,0.15550,0.3010,0.09060,malignant
7,27.420,26.27,186.90,2501.0,0.10840,0.19880,0.36350,0.16890,0.2061,0.05623,...,31.37,251.20,4254.0,0.1357,0.4256,0.68330,0.26250,0.2641,0.07427,malignant
8,14.400,26.99,92.25,646.1,0.06995,0.05223,0.03476,0.01737,0.1707,0.05433,...,31.98,100.40,734.6,0.1017,0.1460,0.14720,0.05563,0.2345,0.06464,benign
9,11.600,18.36,73.88,412.7,0.08508,0.05855,0.03367,0.01777,0.1516,0.05859,...,24.02,82.68,495.1,0.1342,0.1808,0.18600,0.08288,0.3210,0.07863,benign


In [15]:
# 결과 저장.
df_test.to_csv('data/breast_cancer_deep_result.csv')